In [2]:
import numpy
import os
import glob
import matplotlib.pyplot as plt
from skimage import transform as tf
import math 
%matplotlib inline

In [14]:
from keras.utils import np_utils

from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K

In [4]:
dataLocation = ['/Users/gattia/Data/mri/ski10Dataset/']
trainingImages = 'train_images.npy'
trainingLabels = 'train_labels.npy'

img_rows = 448 
img_cols = 352

smooth = 1.

In [5]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [20]:
def get_unet():
    inputs = Input((1, img_rows/2, img_cols/2))
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model 

In [21]:
def preprocess(labels):
    labels[numpy.where(labels==1)] = 0
    labels[numpy.where(labels==3)] = 0
    labels[numpy.where(labels==2)] = 1
    labels[numpy.where(labels==4)] = 1
    return labels

In [22]:
def import_and_process(trainingImagesLocation, testingImagesLocation):
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    trainImages = numpy.load(trainingImagesLocation)
    trainLabels = numpy.load(testingImagesLocation)
    trainLabels = preprocess(trainLabels)
    return(trainLabels, trainImages)

In [23]:
trainLabels, trainImages = import_and_process(dataLocation[0] + trainingImages, dataLocation[0] + trainingLabels)

------------------------------
Loading and preprocessing train data...
------------------------------


In [24]:
def train(dataLocation):
    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    model_checkpoint = ModelCheckpoint(dataLocation + 'uNet.hdf5', monitor='loss', save_best_only=True)
    
    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(trainImages, trainLabels, batch_size=32, nb_epoch=10, verbose=1, shuffle=True,
              callbacks=[model_checkpoint])
    
    

In [25]:
train(dataLocation[0])

------------------------------
Creating and compiling model...
------------------------------
------------------------------
Fitting model...
------------------------------
Epoch 1/10
  352/17428 [..............................] - ETA: 27497s - loss: -0.0018 - dice_coef: 0.0018

KeyboardInterrupt: 